Cargo los datos sobre temperaturas durante el año 2021 de Barcelona desde la estación de Drassanes que tiene un criterio de temperatura a nivel ciudad.

In [12]:
import pandas as pd

# Cargo el archivo csv
df_temp_2022 = pd.read_csv('2022raw.csv', header=0, delimiter= ";")

# Muestro el DataFrame
df_temp_2022.head()

,year,month,day,prec,tmax,tmin
0,2022,1,1,0.0,22.1,14.9
1,2022,1,2,0.0,16.4,9.6
2,2022,1,3,0.0,17.1,12.2
3,2022,1,4,10.1,21.1,10.1
4,2022,1,5,0.4,11.0,5.7


Creo la columna fecha teniendo como referencia las columnas `year`, `month`, `day`

In [13]:
# Creo la columna 'fecha' combinando las columnas renombradas
df_temp_2022['fecha'] = pd.to_datetime(df_temp_2022[['year', 'month', 'day']])

# Formateo la columna 'fecha' a 'yyyy-mm-dd'
df_temp_2022['fecha'] = df_temp_2022['fecha'].dt.strftime('%Y-%m-%d')

df_temp_2022.head()

,year,month,day,prec,tmax,tmin,fecha
0,2022,1,1,0.0,22.1,14.9,2022-01-01
1,2022,1,2,0.0,16.4,9.6,2022-01-02
2,2022,1,3,0.0,17.1,12.2,2022-01-03
3,2022,1,4,10.1,21.1,10.1,2022-01-04
4,2022,1,5,0.4,11.0,5.7,2022-01-05


El dataframe está compuesto por 365 filas y 7 columnas

In [14]:
df_temp_2022.shape 

(365, 7)

Elimino las columnas `year`, `month`, `day` al tener ya la columna fecha

In [15]:
df_temp_2022= df_temp_2022.drop(columns=['year', 'month','day'])

df_temp_2022.head()

,prec,tmax,tmin,fecha
0,0.0,22.1,14.9,2022-01-01
1,0.0,16.4,9.6,2022-01-02
2,0.0,17.1,12.2,2022-01-03
3,10.1,21.1,10.1,2022-01-04
4,0.4,11.0,5.7,2022-01-05


Reviso los tipos de archivo del dataframe

In [16]:
df_temp_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   prec    365 non-null    float64
 1   tmax    365 non-null    float64
 2   tmin    365 non-null    float64
 3   fecha   365 non-null    object 
dtypes: float64(3), object(1)
memory usage: 11.5+ KB


Realizo la transformación de la columna `fecha` al tipo `datetime`

In [17]:
df_temp_2022['fecha']= pd.to_datetime(df_temp_2022['fecha'], format='%Y-%m-%d')
df_temp_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   prec    365 non-null    float64       
 1   tmax    365 non-null    float64       
 2   tmin    365 non-null    float64       
 3   fecha   365 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(3)
memory usage: 11.5 KB


Compruebo si existen valores nulos en el dataframe.

In [18]:
df_temp_2022.isnull().sum()

prec     0
tmax     0
tmin     0
fecha    0
dtype: int64

Voy a crear la columna `tmed` para tener la media del día

In [19]:
df_temp_2022['tmed']= (df_temp_2022['tmin'] + df_temp_2022['tmax']) / 2
df_temp_2022.head()

,prec,tmax,tmin,fecha,tmed
0,0.0,22.1,14.9,2022-01-01,18.50
1,0.0,16.4,9.6,2022-01-02,13.00
2,0.0,17.1,12.2,2022-01-03,14.65
3,10.1,21.1,10.1,2022-01-04,15.60
4,0.4,11.0,5.7,2022-01-05,8.35


Agrego la columna ciudad con "Barcelona"

In [20]:
df_temp_2022['ciudad'] = "Barcelona"
df_temp_2022.head()

,prec,tmax,tmin,fecha,tmed,ciudad
0,0.0,22.1,14.9,2022-01-01,18.50,Barcelona
1,0.0,16.4,9.6,2022-01-02,13.00,Barcelona
2,0.0,17.1,12.2,2022-01-03,14.65,Barcelona
3,10.1,21.1,10.1,2022-01-04,15.60,Barcelona
4,0.4,11.0,5.7,2022-01-05,8.35,Barcelona


- Reordeno las columnas

In [21]:
df_temp_2022= df_temp_2022[['fecha', 'ciudad', 'tmin', 'tmax', 'tmed', 'prec']]
df_temp_2022.head()


,fecha,ciudad,tmin,tmax,tmed,prec
0,2022-01-01,Barcelona,14.9,22.1,18.50,0.0
1,2022-01-02,Barcelona,9.6,16.4,13.00,0.0
2,2022-01-03,Barcelona,12.2,17.1,14.65,0.0
3,2022-01-04,Barcelona,10.1,21.1,15.60,10.1
4,2022-01-05,Barcelona,5.7,11.0,8.35,0.4


Exporto el dataframe a un archivo en formato `csv`

In [22]:
df_temp_2022.to_csv('temp_bcn_2022.csv', index= False)

Resumen del dataframe creado:

- Se agregó la columna `fecha` tomando como referencia las columnas `year`, ` month` y `day`, que posteriormente se eliminaron.
- Se agregó la columna `tmed` calculando la media de las columnas `tmin` y `tmax`.
- Se agregó la columna `ciudad` con el registro `Barcelona`
- No se encontraron valores nulos.
- Se reordenaron las columnas
- Se ha exportado el dataframe final a un archivo en formato `csv`